In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

In [32]:
def NSW(xs):
    return -np.prod(xs*np.array([1,2,400]))

In [33]:
xs = np.array([1/3, 1/3, 1/3])
NSW(xs)

-29.629629629629623

In [34]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

In [35]:
res = minimize(NSW, start_pos, method='SLSQP', bounds=bnds ,constraints=cons)
res

     fun: -29.629629627978957
     jac: array([-88.8881228 , -88.88927197, -88.88927197])
 message: 'Optimization terminated successfully'
    nfev: 26
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.33333621, 0.3333319 , 0.3333319 ])

In [7]:
alpha = 2
def F(x):
    costs = [1., 2., 3.]
    vals  = np.array([costs[i]*np.power(x[i], 1+1/alpha) for i in range(len(costs))])
    return np.sum(vals/(1+1/alpha))

In [8]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 0.5714285714285716
     jac: array([0.85714287, 0.85714287, 0.85714287])
 message: 'Optimization terminated successfully'
    nfev: 30
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.73469389, 0.18367347, 0.08163265])

In [9]:
def theos(costs):
    den = np.sum([np.power(costs[i], -alpha) for i in range(len(costs))])
    return [np.power(costs[i], -alpha) / den for i in range(len(costs))]

In [10]:
theos([1.,2.,3.])

[0.7346938775510203, 0.18367346938775508, 0.08163265306122448]

# normal tullock

In [15]:
alpha = 2
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(x[i], 1+1/alpha)/vs[i] for i in range(len(vs))])
    return np.sum(vals)

In [16]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 0.2672612419124245
     jac: array([0.40089187, 0.40089186, 0.40089187])
 message: 'Optimization terminated successfully'
    nfev: 32
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([0.07142856, 0.28571428, 0.64285716])

In [17]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [18]:
theos([1.,2.,3.])

[0.07142857142857142, 0.2857142857142857, 0.6428571428571429]

In [158]:
# other direction

In [19]:
alpha = 2
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(x[i], 1-1/alpha)*vs[i] for i in range(len(vs))])
    return -np.sum(vals)

In [20]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: -3.7416573867739413
     jac: array([-1.87082866, -1.87082866, -1.87082866])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 39
    njev: 39
  status: 0
 success: True
       x: array([0.07142857, 0.28571428, 0.64285715])

In [21]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [22]:
theos([1.,2.,3.])

[0.07142857142857142, 0.2857142857142857, 0.6428571428571429]

In [158]:
# v_i^a

In [182]:
alpha = 2
def F(x):
    vs    = [1., 2., 3.]
    vals  = np.array([np.power(vs[i], alpha)*x[i] for i in range(len(vs))])
    return -np.sum(vals)

In [183]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds ,constraints=cons, tol=1e-16)
res

     fun: 1.000000000000015
     jac: array([0.99999999, 4.        , 9.        ])
 message: 'Positive directional derivative for linesearch'
    nfev: 64
     nit: 10
    njev: 6
  status: 8
 success: False
       x: array([1.00000000e+00, 3.78882784e-15, 0.00000000e+00])

In [177]:
def theos(vals):
    den = np.sum([np.power(vals[i], alpha) for i in range(len(vals))])
    return [np.power(vals[i], alpha) / den for i in range(len(vals))]

In [178]:
theos([1.,2.,3.])

[0.16653664532144477, 0.33330423948674176, 0.5001591151918134]

# Proportional

In [28]:
def F(x):
    vs    = [1., 2., 4.]
    vals  = np.array([np.log(x[i])*vs[i] for i in range(len(vs))])
    return -np.sum(vals)

In [29]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds, constraints=cons, tol=1e-16)
res

     fun: 6.68989923778774
     jac: array([-6.99999988, -6.99999976, -6.99999988])
 message: 'Optimization terminated successfully'
    nfev: 73
     nit: 15
    njev: 15
  status: 0
 success: True
       x: array([0.14285714, 0.28571429, 0.57142857])

In [30]:
1/7, 2/7, 4/7

(0.14285714285714285, 0.2857142857142857, 0.5714285714285714)

# WTA

In [36]:
def F(x):
    vs    = [1., 2., 4.]
    return -np.dot(x,vs)

In [37]:
start_pos = [0.8, 0.1, 0.1]
cons = ({'type': 'eq', 'fun': lambda x:  1 - sum(x)})
bnds = tuple((0,1) for x in start_pos)

res = minimize(F, start_pos, method='SLSQP', bounds=bnds, constraints=cons, tol=1e-16)
res

     fun: -4.0
     jac: array([-1., -2., -4.])
 message: 'Optimization terminated successfully'
    nfev: 8
     nit: 6
    njev: 2
  status: 0
 success: True
       x: array([0., 0., 1.])